In [1]:
sentence1 = "Hi my name is Abhisek. I am a Data Scientist"

In [3]:
sentence1.split()

['Hi', 'my', 'name', 'is', 'Abhisek.', 'I', 'am', 'a', 'Data', 'Scientist']

# NER Data Formats
Pandas DataFrame:

A DataFrame containing the 3 columns sentence_id, words, labels. Each value in words will have a corresponding labels value. The sentence_id determines which words belong to a given sentence. I.e. the words from the same sequence should be assigned the same unique sentence_id.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/ner_datasetreference.csv',
                 encoding='unicode_escape')
df.head(30)

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O
5,NaN,through,IN,O
6,NaN,London,NNP,B-geo
7,NaN,to,TO,O
8,NaN,protest,VB,O
9,NaN,the,DT,O


In [4]:
# Filling the nan values 
df.ffill(axis=0, inplace=True)
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O


In [5]:
# Encode the sentence # column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['Sentence #'] = le.fit_transform(df['Sentence #'])

In [6]:
df.rename(columns={'Sentence #':'Sentence_id'}, inplace=True)
df.head()

,Sentence_id,Word,POS,Tag
0,0,Thousands,NNS,O
1,0,of,IN,O
2,0,demonstrators,NNS,O
3,0,have,VBP,O
4,0,marched,VBN,O


In [7]:
labels = df['Tag'].unique().tolist()
print(labels)

['O', 'B-geo', 'B-gpe', 'B-per', 'I-geo', 'B-org', 'I-org', 'B-tim', 'B-art', 'I-art', 'I-per', 'I-gpe', 'I-tim', 'B-nat', 'B-eve', 'I-eve', 'I-nat']


In [8]:
X = df.iloc[:,0:2]
y = df.iloc[:,3]

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)
print("Shape of X_train: ", X_train.shape,
      "\nShape of y_train: ", y_train.shape,
      "\nShape of X_test: ", X_test.shape,
      "\nShape of y_test: ", y_test.shape)

Shape of X_train:  (943717, 2) 
Shape of y_train:  (943717,) 
Shape of X_test:  (104858, 2) 
Shape of y_test:  (104858,)


In [10]:
type(y_train)

pandas.core.series.Series

In [30]:
# Creating format of simpletransformers
train_data = pd.concat([X_train, y_train], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)
test_data.rename(columns={'Sentence_id': 'sentence_id', 'Word':'words', 'Tag':'labels'}, inplace=True)
train_data.rename(columns={'Sentence_id': 'sentence_id', 'Word':'words', 'Tag':'labels'}, inplace=True)
test_data

,sentence_id,words,labels
875680,33351,second,O
1046906,42087,country,O
646861,21739,150,O
704385,24664,approve,O
798051,29424,interview,O
...,...,...,...
859478,32527,work,O
637076,21253,been,O
86900,32779,Mr.,B-per
1029806,41211,the,O


In [1]:
!pip install simpletransformers

In [12]:
from simpletransformers.ner import NERArgs, NERModel

In [31]:
args = NERArgs()
args.num_train_epochs = 1
args.train_batch_size = 32
args.overwrite_output_dir = True
args.learning_rate = 1e-4
#args.labels_list =labels

In [32]:
model = NERModel("bert", "bert-base-uncased", labels=labels, args=args)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

In [33]:
model.train_model(train_data=train_data, eval_data=test_data, acc='accuracy_score')

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/1499 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


(1499, 0.41574711520088126)

In [34]:
# Evaluate the model
result, model_outputs, preds_list = model.eval_model(test_data)

  0%|          | 0/3 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/5177 [00:00<?, ?it/s]

In [35]:
result

{'eval_loss': 0.4824060571694171,
 'precision': 0.3947272727272727,
 'recall': 0.13820103125596792,
 'f1_score': 0.2047244094488189}

In [48]:
query = df[df['Sentence_id']==10]['Word'].tolist()
#data[data['Sentence #']=='Sentence: 1']['Word'].tolist()

In [49]:
# Make predictions with the model
query = ' '.join(query)
predictions, raw_outputs = model.predict([query])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [50]:
predictions

[[{'In': 'B-geo'},
  {'Beirut': 'B-geo'},
  {',': 'O'},
  {'a': 'O'},
  {'string': 'O'},
  {'of': 'O'},
  {'officials': 'O'},
  {'voiced': 'O'},
  {'their': 'O'},
  {'anger': 'O'},
  {',': 'O'},
  {'while': 'O'},
  {'at': 'O'},
  {'the': 'O'},
  {'United': 'B-geo'},
  {'Nations': 'B-geo'},
  {'summit': 'O'},
  {'in': 'O'},
  {'New': 'B-geo'},
  {'York': 'B-geo'},
  {',': 'O'},
  {'Prime': 'B-geo'},
  {'Minister': 'B-geo'},
  {'Fouad': 'B-geo'},
  {'Siniora': 'B-geo'},
  {'said': 'O'},
  {'the': 'O'},
  {'Lebanese': 'B-geo'},
  {'people': 'O'},
  {'are': 'O'},
  {'resolute': 'O'},
  {'in': 'O'},
  {'preventing': 'O'},
  {'such': 'O'},
  {'attempts': 'O'},
  {'from': 'O'},
  {'destroying': 'O'},
  {'their': 'O'},
  {'spirit': 'O'},
  {'.': 'O'}]]

In [51]:
df[df['Sentence_id']==10]['Tag'].tolist()

['O',
 'B-geo',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-org',
 'I-org',
 'O',
 'O',
 'B-geo',
 'I-geo',
 'O',
 'B-per',
 'O',
 'B-per',
 'I-per',
 'O',
 'O',
 'B-gpe',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']